In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import KFold
from tensorflow import keras

In [ ]:
SurfaceData=pd.read_csv('SurfaceData.csv',delimiter=";")

In [ ]:
for i in range(len(SurfaceData)):
    SurfaceData["bins"][i]=np.fromstring(SurfaceData["bins"][i][1:-2],sep=',')
    SurfaceData["n0"][i]=np.fromstring(SurfaceData["n0"][i][1:-2],sep=',')
    SurfaceData["Qquerys"][i]=np.fromstring(SurfaceData["Qquerys"][i][1:-2],sep=',')
    SurfaceData["PSquerys"][i]=np.fromstring(SurfaceData["PSquerys"][i][1:-2],sep=',')

In [ ]:
SurfaceData_Simulated=SurfaceData[SurfaceData["DU"].notna()]#Select labeled surfaces
SurfaceData_Simulated=SurfaceData_Simulated[SurfaceData["DU"]>=6]#select fully rough
SurfaceData_Pool=SurfaceData[SurfaceData["DU"].isna()]#Transerring unlabeled surfaces in repository
ListOfLabledID=SurfaceData_Simulated["Surface_ID"].to_numpy()
DUTrain=SurfaceData_Simulated["kr"].to_numpy()#Get labels
SurfacePoolID=SurfaceData_Pool["Surface_ID"].to_numpy()

Iteration_num=50 # number of NN members
SurfaceTrain=[]
SurfaceReal=[]

In [ ]:
for i in ListOfLabledID.astype(int):
    SurfaceTrain_row=[SurfaceData.iloc[i].kt/SurfaceData.iloc[i].K_99,SurfaceData.iloc[i].lambda0_k99,SurfaceData.iloc[i].lambda1_k99]
    SurfaceTrain_row.extend(SurfaceData.iloc[i].n0)
    SurfaceTrain_row.extend(SurfaceData.iloc[i].PSquerys)
    SurfaceTrain.append(SurfaceTrain_row)

In [ ]:
space = [
    Integer(1, 256, name='n_neurons_0'),
    Integer(1, 256, name='n_neurons_1'),
    Integer(1, 256, name='n_neurons_2'),
    Categorical(['sigmoid','linear',"tanh", 'relu','leakyrelu'], name='activation'),
    Real(1e-4, 1e-1, prior='log-uniform', name='lr'),
    Real(1e-4, 1e-1, prior='log-uniform', name='regularization')
]#Specify to be tuned hyperparameters and the range
@use_named_args(dimensions=space)

def build_model(**space):
    model = keras.Sequential()
    # Add input layer; images have one channel (102, 302, 1)
    input_shape = 63
    if space['activation']=='leakyrelu':
        model.add(keras.layers.Dense(int(space['n_neurons_0']),input_shape=(input_shape,),activation=keras.layers.LeakyReLU(alpha=0.1),kernel_regularizer=keras.regularizers.l2(space['regularization'])))
    else:
        model.add(keras.layers.Dense(int(space['n_neurons_0']),input_shape=(input_shape,),activation=keras.layers.Activation(space['activation']),kernel_regularizer=keras.regularizers.l2(space['regularization'])))
    for i in range(2):
        model.add(keras.layers.Dense(int(space['n_neurons_'+str(i+1)]),kernel_regularizer=keras.regularizers.l2(space['regularization'])))
        if space['activation'] == 'leakyrelu':
            model.add(keras.layers.LeakyReLU(alpha=0.1))
        else:
            model.add(keras.layers.Activation(space['activation']))
    # Add output layer
    model.add(keras.layers.Dense(1,kernel_regularizer=keras.regularizers.l2(space['regularization'])))
    if space['activation'] == 'leakyrelu':
        model.add(keras.layers.LeakyReLU(alpha=0.1))
    else:
        model.add(keras.layers.Activation(space['activation']))
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(space['lr']), loss=keras.losses.MeanSquaredError(),
                  metrics=[keras.metrics.MeanSquaredError(),keras.metrics.MeanAbsolutePercentageError(name="MAPE")])
    return model
def lr_scheduler(epoch, learning_rate):
    lr = learning_rate
    if epoch == 1000:
        learning_rate = lr*0.5
        return learning_rate
    elif epoch == 1500:
        learning_rate = lr*0.5
        return learning_rate
    elif epoch == 1800:
        learning_rate = lr*0.5
        return learning_rate
    else:
        learning_rate = lr
        return learning_rate
    
new_lr = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)
def output_val_loss_BO(epoch,logs):
    print('the keys are:{}'.format(logs.keys))
def objective(space):
    kfold=KFold(n_splits=10,shuffle=True)
    fold_num=1
    val_mse_k_fold=np.zeros(10)
    print('Current space')
    print(space)
    for train_id,val_id in kfold.split(SurfaceTrain,DUTrain):

        model = build_model(space)
        #start_epoch = time.time()
    # ckpt_filepath = './tmp_cnn_v1/cnn_ckpt/model_{epoch:03d}_{val_loss:.4f}.h5'
    # ckpt = ModelCheckpoint(ckpt_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300)
        X_train=SurfaceTrain[train_id]
        y_train=DUTrain[train_id]
        X_val=SurfaceTrain[val_id]
        y_val=DUTrain[val_id]
        history = model.fit(X_train, y_train, epochs=2000,
                        validation_data=(X_val, y_val),
                        callbacks=[new_lr,es],verbose=0)
        #print('---- training time for each evaluation %s seconds ----' % (time.time() - start_epoch))
        
        train_loss, val_mse = history.history['loss'], history.history['val_MAPE']
        val_mse_k_fold[fold_num-1]=np.min(val_mse)
        fold_num+=1
    val_mse_mean=np.mean(val_mse_k_fold)
    return np.min(val_mse_mean)

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)   
print("-----------Run BO to find the optimal hps------------")
result = gp_minimize(func=objective, dimensions=space, n_calls=60, acq_func='gp_hedge', n_restarts_optimizer=5,verbose=1)
print("Best parameters: ", result.x)
print("Minimum val loss: ", result.fun)
    
ToSave={"n_neurons_0":result.x[0],"n_neurons_1":result.x[1],"n_neurons_2":result.x[2],'activation':result.x[3],"lr":result.x[4],"regularization":result.x[5]}
df=pd.DataFrame(ToSave,index=[0])
df.to_csv('./Hyperparameters_kfold.csv',index=False,sep=';')


In [ ]:
ToSave={"n_neurons_0":result.x[0],"n_neurons_1":result.x[1],"n_neurons_2":result.x[2],'activation':result.x[3],"lr":result.x[4],"regularization":result.x[5]}
df=pd.DataFrame(ToSave,index=[0])
df.to_csv('./Hyperparameters_BO.csv',index=False,sep=';')#Output hyperparameters in csv file

In [ ]:
val_loss_history=result.func_vals
plt.plot(val_loss_history,color=[0,130/255,150/255],linewidth=3)
plt.plot(np.argmin(val_loss_history),np.min(val_loss_history),'ro')
plt.legend(["BO","selected","Previous"])
plt.xlabel("BO iterations")
plt.ylabel("val_loss")
plt.show()